In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib.pyplot as plt
import pickle
import copy
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.datasets import cifar10
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils.visualize_util import model_to_dot, plot
from keras import backend as K
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN 4007)


In [3]:
WORKSPACE_PATH = '/home/takeyama/pywork/ipython/2016-06-27'

In [4]:
RawDataPath=WORKSPACE_PATH+'/raw/'

In [5]:
import processing

センサデータを読み込む

In [6]:
dic1=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor1")
dic2=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor2")
dic3=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor3")
dic4=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor4")
dic5=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor5")
dic6=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor6")
dic7=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor7")

windows関数を作成する。
要求仕様
* 入力データは2次元配列　raw 時間、colm　データの種類（加速度X,加速度Y,・・・）
* 出力データは4次元配列　numberOfWindow * windowWidth * kindOfData * 
# how to use
1. Set Data(1darray)
case 1  
    seed.append(AccX)  
    seed.append(AccY)  
    seed.append(AccZ)  
2. Making window
w = build(seed)  
w -> [ [AccX],[AccY],[AccZ] ]  
window = compile(w)  
window -> 
          [ [[AccX],[AccY],[AccZ]],  
            [[AccX],[AccY],[AccZ]],  
            [[AccX],[AccY],[AccZ]], ....]

In [7]:
class Window:
    """Input dictionary
       Ouput Windows
    """         
    def __init__(self):                  
        self.seed ={}
    
    def SetData(self, RegistDataName, RegistData):
        if RegistDataName in self.seed:
            print RegistDataName+' has been registed before'
            return -1
        
        self.seed[RegistDataName] = RegistData
        print RegistDataName+' is registed now'
    
    def _Build(self):
        keys = self.seed.keys()
        array = self.seed[keys[0]]
        
        for k in keys[1:]:
            array = np.vstack( (array,self.seed[k]))
        print 'Build Complete'
        return array.T
    
    def Compile(self,windowWidth,overlap):
        source = self._Build()
        print source.shape
        sourceRows,sourceColum=source.shape
        offset = len(source)%windowWidth
        if offset % 2 == 1:
            offset=offset/2 +1
        else:
            offset=offset/2
        # first array
        start = offset
        goal = start+windowWidth
        window = source[start:goal].T
        
        for i in range(1,len(source)/windowWidth):
            start = start+ int(windowWidth*overlap)
            goal = start+windowWidth
            adding = source[start:goal].T
            window = np.dstack((window,adding))
        return window.T

In [8]:
w = Window()

In [9]:
w.SetData('dic1-AccX',dic1['AccX'])
w.SetData('dic1-AccY',dic1['AccY'])
w.SetData('dic1-AccZ',dic1['AccZ'])

dic1-AccX is registed now
dic1-AccY is registed now
dic1-AccZ is registed now


In [10]:
w.SetData('dic2-AccX',dic2['AccX'])
w.SetData('dic2-AccY',dic2['AccY'])
w.SetData('dic2-AccZ',dic2['AccZ'])

dic2-AccX is registed now
dic2-AccY is registed now
dic2-AccZ is registed now


In [11]:
w.SetData('dic3-AccX',dic3['AccX'])
w.SetData('dic3-AccY',dic3['AccY'])
w.SetData('dic3-AccZ',dic3['AccZ'])

dic3-AccX is registed now
dic3-AccY is registed now
dic3-AccZ is registed now


In [12]:
w.SetData('dic4-AccX',dic4['AccX'])
w.SetData('dic4-AccY',dic4['AccY'])
w.SetData('dic4-AccZ',dic4['AccZ'])

dic4-AccX is registed now
dic4-AccY is registed now
dic4-AccZ is registed now


In [13]:
w.SetData('dic5-AccX',dic5['AccX'])
w.SetData('dic5-AccY',dic5['AccY'])
w.SetData('dic5-AccZ',dic5['AccZ'])

dic5-AccX is registed now
dic5-AccY is registed now
dic5-AccZ is registed now


In [14]:
w.SetData('dic6-AccX',dic6['AccX'])
w.SetData('dic6-AccY',dic6['AccY'])
w.SetData('dic6-AccZ',dic6['AccZ'])

dic6-AccX is registed now
dic6-AccY is registed now
dic6-AccZ is registed now


In [15]:
w.SetData('dic7-AccX',dic7['AccX'])
w.SetData('dic7-AccY',dic7['AccY'])
w.SetData('dic7-AccZ',dic7['AccZ'])

dic7-AccX is registed now
dic7-AccY is registed now
dic7-AccZ is registed now


In [16]:
window = w.Compile(windowWidth=16,overlap=0.5)

Build Complete
(254742, 21)


出来上がったwindowの形をみれば、Mnistを模倣したほうがいいかもしれない  
というのは、windowの集合を見るとMnistに近いものになっていた  
Mnistの訓練データは[sample, 28,28]  
一方、出来上がったwindowは[sample,windowWidth,data]  
また、cifar10は[sample,32,32,3]  


In [17]:
window.shape

(15921, 16, 21)

kerasとMnistに関して、入力層をどのようにやっているのかわからないので実際にデータをダウンロードしてどのような構造でダウンロードして、入力層に与えるときに何をしているかをはっきりさせる

In [18]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

In [19]:
x_train.shape

(60000, 28, 28)

In [20]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print x_train.shape
print x_test.shape

(60000, 784)
(10000, 784)


以上から、(60000, 28, 28)の3次元でダウンロードして(60000, 784)に変換している

In [25]:
window_train = window.reshape((len(window),np.prod(window.shape[1:])))
#window_train = window_train[:12000]
window_test = window.reshape((len(window),np.prod(window.shape[1:])))
#window_test = window_test[12001:]

In [26]:
print window_train.shape

(15921, 336)


define model

In [30]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

shapeNum=window_train.shape[1]

# this is our input placeholder
input_img = Input(shape=(shapeNum,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(shapeNum, activation='tanh')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(window_train, window_train,
                nb_epoch=50,
                batch_size=shapeNum/4,
                shuffle=True,
                validation_data=(window_test, window_test))

Train on 15921 samples, validate on 15921 samples
Epoch 1/50
15921/15921 [==============================] - 3s - loss: -4.5150 - val_loss: -5.1101
Epoch 2/50
15921/15921 [==============================] - 3s - loss: -5.1615 - val_loss: -5.1990
Epoch 3/50
15921/15921 [==============================] - 3s - loss: -5.2200 - val_loss: -5.2327
Epoch 4/50
15921/15921 [==============================] - 3s - loss: -5.2391 - val_loss: -5.2462
Epoch 5/50
15921/15921 [==============================] - 3s - loss: -5.2529 - val_loss: -5.2616
Epoch 6/50
15921/15921 [==============================] - 3s - loss: -5.2675 - val_loss: -5.2744
Epoch 7/50
15921/15921 [==============================] - 3s - loss: -5.2916 - val_loss: -5.3037
Epoch 8/50
15921/15921 [==============================] - 3s - loss: -5.3099 - val_loss: -5.3155
Epoch 9/50
15921/15921 [==============================] - 3s - loss: -5.3214 - val_loss: -5.3261
Epoch 10/50
15921/15921 [==============================] - 3s - loss: -5.3291

In [31]:
encoded_imgs = encoder.predict(window_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [44]:
window_train =window_train.reshape(15921,16, 21)

In [47]:
decoded_imgs = decoded_imgs.reshape(15921,16, 21)